In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import networkx as nx
import random


#print(os.listdir("../input"))

pd.set_option("max_colwidth",1000000)
pd.set_option('max_columns', 15000)

# Read dataset
survey_2019 = pd.read_csv(r'C:\\Users\\lan\\Desktop\\StackOverflowSurvey\\developer_survey_2019\\survey_results_public.csv')
schema = pd.read_csv(r'C:\\Users\\lan\\Desktop\\StackOverflowSurvey\\developer_survey_2019\\survey_results_schema.csv')

# The transpose data set should have 88883-7548=81335 observations.
#df = pd.get_dummies(survey_2019['DevType'].str.split(';', expand=True).stack()).sum(level=0)

# 工作环境对编程语言的影响（工作地点在家和在公司对编程语言的影响）
# 不同国家编程人员的工作时间
# 工作时间对满意度的影响

In [22]:
df = survey_2019[survey_2019['DevType'] == '']

Index(['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource',
       'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor',
       'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode',
       'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney',
       'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz',
       'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc',
       'CompTotal', 'CompFreq', 'ConvertedComp', 'WorkWeekHrs', 'WorkPlan',
       'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev',
       'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat',
       'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife'

In [2]:
mixed_data_scientist = survey_2019[survey_2019['DevType'].str.contains("Data scientist",na = False)
                                  & (survey_2019['DevType']!='Data scientist or machine learning specialist')]
pure_data_scientist = survey_2019[survey_2019['DevType']=='Data scientist or machine learning specialist']

In [3]:
df = pd.concat([mixed_data_scientist,pure_data_scientist])

In [4]:
def total_count(df, col1):
    '''
    INPUT:
    df - the pandas dataframe you want to search
    col1 - the column name you want to look through
    
    OUTPUT:
    new_df - a dataframe shows the percentage account for the total observation. 
    '''
    new_df = df[col1].str.split(';', expand=True).stack().value_counts(dropna=False).reset_index()
    new_df.rename(columns={0: 'count'}, inplace=True)
    new_df['percentage'] = new_df['count']/np.sum(df[col1].notnull())
    return new_df

In [30]:
df.shape

(6460, 85)

In [48]:
#性别占比
df['Gender'].value_counts()

Man                                                            5656
Woman                                                           486
Non-binary, genderqueer, or gender non-conforming                50
Woman;Man                                                        21
Man;Non-binary, genderqueer, or gender non-conforming            19
Woman;Non-binary, genderqueer, or gender non-conforming          12
Woman;Man;Non-binary, genderqueer, or gender non-conforming      10
Name: Gender, dtype: int64

In [46]:
男：5656 女：486 其他：112

In [20]:
#年龄占比
#len(df[(df['Age'] > 0) & (df['Age'] < 16)]) 25
#len(df[(df['Age'] > 15) & (df['Age'] < 26)]) 1818
#len(df[(df['Age'] > 25) & (df['Age'] < 36)]) 2645
#len(df[(df['Age'] > 35) & (df['Age'] < 46)]) 905
#len(df[(df['Age'] > 45)]) 370

25

In [6]:
from pyecharts.charts import Polar
def polar_radiusaxis() -> Polar:
    c = (
        Polar()
        .add_schema(
            radiusaxis_opts = opts.RadiusAxisOpts(type_="category"),
            angleaxis_opts = opts.AngleAxisOpts(max_ = 5763)
        )
        .add("0-15岁", [25], type_="bar",)
        .add("16-25岁", [1818], type_="bar")
        .add("26-35岁", [2645], type_="bar")
        .add("36-45岁", [905], type_="bar")
        .add("46岁以上", [370], type_="bar")
        .set_global_opts(title_opts=opts.TitleOpts(title="Polar-RadiusAxis"))
    )
    return c
polar_radiusaxis().render_notebook()

In [9]:
#教育水平
df['EdLevel'].value_counts()

Master’s degree (MA, MS, M.Eng., MBA, etc.)                                           2301
Bachelor’s degree (BA, BS, B.Eng., etc.)                                              2261
Other doctoral degree (Ph.D, Ed.D., etc.)                                              780
Some college/university study without earning a degree                                 421
Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)     336
Associate degree                                                                       102
Professional degree (JD, MD, etc.)                                                      74
Primary/elementary school                                                               62
I never completed any formal education                                                  34
Name: EdLevel, dtype: int64

In [11]:
#雷达图————————————————————————
#1、平均工作时长
df['WorkWeekHrs'].value_counts().mean()

51.861702127659576

In [13]:
#2、工作满意度 百分比
df['JobSat'].value_counts()

Very satisfied                        1890
Slightly satisfied                    1784
Slightly dissatisfied                  687
Neither satisfied nor dissatisfied     646
Very dissatisfied                      370
Name: JobSat, dtype: int64

In [49]:
#和父辈比有着更好的生活
df['BetterLife'].value_counts()

Yes    4109
No     2213
Name: BetterLife, dtype: int64

In [105]:
new_df = df['YearsCodePro'].str.replace("More than 50 years","51").replace("Less than 1 year","0.5").astype(float).value_counts().reset_index()
new_df

,index,YearsCodePro
0,2.0,687
1,3.0,671
2,5.0,508
3,4.0,490
4,1.0,372
5,10.0,338
6,6.0,309
7,0.5,299
8,8.0,255
9,7.0,249


In [107]:
#平均从业年龄
np.average(new_df['YearsCodePro'],weights = new_df['index'])

37.58097251585624

In [85]:
df['YearsCodePro'].value_counts()

2                     687
3                     671
5                     508
4                     490
1                     372
10                    338
6                     309
Less than 1 year      299
8                     255
7                     249
9                     163
20                    161
12                    136
15                    129
11                    111
13                     95
14                     75
18                     75
25                     65
16                     61
30                     46
22                     42
23                     39
19                     36
21                     36
17                     35
24                     24
28                     20
32                     15
35                     15
26                     15
27                     11
40                     11
29                     10
33                      7
More than 50 years      7
45                      6
31                      6
34          

In [52]:
#工作遇到的困难
total_count(df,'WorkChallenge')

,index,count,percentage
0,Being tasked with non-development work,2008,0.386600
1,Distracting work environment,2000,0.385060
2,Meetings,1920,0.369657
3,Not enough people for the workload,1775,0.341740
4,Inadequate access to necessary tools,1322,0.254524
5,Lack of support from management,1322,0.254524
6,"Non-work commitments (parenting, school work, hobbies, etc.)",1098,0.211398
7,Time spent commuting,1015,0.195418
8,Toxic work environment,997,0.191952


In [77]:
#平均工资
np.average(df['ConvertedComp'].dropna())

,index,ConvertedComp
0,2000000.0,69
1,120000.0,50
2,150000.0,46
3,100000.0,42
4,1000000.0,40
...,...,...
1853,24024.0,1
1854,17940.0,1
1855,109020.0,1
1856,23280.0,1


In [13]:
np.average(df['ConvertedComp'].dropna())

142191.2698600901

In [12]:
#社交APP偏好
df['SocialMedia'].value_counts()

WhatsApp                    1025
Reddit                       990
YouTube                      932
Facebook                     876
Twitter                      872
LinkedIn                     492
I don't use social media     415
Instagram                    408
Snapchat                      48
WeChat 微信                     43
VK ВКонта́кте                 37
Weibo 新浪微博                     8
Hello                          1
Name: SocialMedia, dtype: int64

In [40]:
from pyecharts.charts import Polar,Line
def line_areastyle_boundary_gap() -> Line:
    c = (
        Line()
        .add_xaxis(['WhatsApp','Reddit','YouTube','Facebook','Twitter','LinkedIn','none','Instagram','Snapchat','WeChat','VK','Weibo'])
        .add_yaxis("社交媒体", [1025,990,932,876,872,492,415,408,48,43,37,8], is_smooth=True)
        .set_series_opts(
            areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            #title_opts=opts.TitleOpts(title="Line-面积图（紧贴 Y 轴）"),
            xaxis_opts=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(rotate=-15),
                axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
                is_scale=False,
                boundary_gap=False,
            ),
        )
    )
    return c
line_areastyle_boundary_gap().render_notebook()

In [5]:
from pyecharts import options as opts
from pyecharts.charts import Radar

#平均工作时长
#平均工资
#平均从业年龄

v1 = [[51.8,142191.2,37.5]]

def radar_base() -> Radar:
    c = (
        Radar()
        .add_schema(
            schema=[
                opts.RadarIndicatorItem(name="平均周工作时长", max_=80),
                opts.RadarIndicatorItem(name="工资", max_=200000),
                opts.RadarIndicatorItem(name="平均从业年龄", max_=50),
                opts.RadarIndicatorItem(name="工资", max_=200000),
                #opts.RadarIndicatorItem(name="平均从业年龄", max_=50),
            ]
        )
        .add("数据程序员", v1)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(title_opts=opts.TitleOpts(title="Radar-基本示例"))
    )
    return c
radar_base().render_notebook()

# 世界程序员的分布
# 世界上第一行代码年龄分布

In [9]:
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line, Page, Pie


def bar_datazoom_slider() -> Bar:
    c = (
        Bar()
        .add_xaxis(Faker.days_attrs)
        .add_yaxis("商家A", Faker.days_values)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Bar-DataZoom（slider-水平）"),
            datazoom_opts=[opts.DataZoomOpts()],
        )
    )
    return c


def line_markpoint() -> Line:
    c = (
        Line()
        .add_xaxis(Faker.choose())
        .add_yaxis(
            "商家A",
            Faker.values(),
            markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="min")]),
        )
        .add_yaxis(
            "商家B",
            Faker.values(),
            markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="Line-MarkPoint"))
    )
    return c


def pie_rosetype() -> Pie:
    v = Faker.choose()
    c = (
        Pie()
        .add(
            "",
            [list(z) for z in zip(v, Faker.values())],
            radius=["30%", "75%"],
            center=["25%", "50%"],
            rosetype="radius",
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add(
            "",
            [list(z) for z in zip(v, Faker.values())],
            radius=["30%", "75%"],
            center=["75%", "50%"],
            rosetype="area",
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="Pie-玫瑰图示例"))
    )
    return c


def grid_mutil_yaxis() -> Grid:
    x_data = ["{}月".format(i) for i in range(1, 13)]
    bar = (
        Bar()
        .add_xaxis(x_data)
        .add_yaxis(
            "蒸发量",
            [2.0, 4.9, 7.0, 23.2, 25.6, 76.7, 135.6, 162.2, 32.6, 20.0, 6.4, 3.3],
            yaxis_index=0,
            color="#d14a61",
        )
        .add_yaxis(
            "降水量",
            [2.6, 5.9, 9.0, 26.4, 28.7, 70.7, 175.6, 182.2, 48.7, 18.8, 6.0, 2.3],
            yaxis_index=1,
            color="#5793f3",
        )
        .extend_axis(
            yaxis=opts.AxisOpts(
                name="蒸发量",
                type_="value",
                min_=0,
                max_=250,
                position="right",
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(color="#d14a61")
                ),
                axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
            )
        )
        .extend_axis(
            yaxis=opts.AxisOpts(
                type_="value",
                name="温度",
                min_=0,
                max_=25,
                position="left",
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(color="#675bba")
                ),
                axislabel_opts=opts.LabelOpts(formatter="{value} °C"),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=1)
                ),
            )
        )
        .set_global_opts(
            yaxis_opts=opts.AxisOpts(
                name="降水量",
                min_=0,
                max_=250,
                position="right",
                offset=80,
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(color="#5793f3")
                ),
                axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
            ),
            title_opts=opts.TitleOpts(title="Grid-多 Y 轴示例"),
            tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        )
    )

    line = (
        Line()
        .add_xaxis(x_data)
        .add_yaxis(
            "平均温度",
            [2.0, 2.2, 3.3, 4.5, 6.3, 10.2, 20.3, 23.4, 23.0, 16.5, 12.0, 6.2],
            yaxis_index=2,
            color="#675bba",
            label_opts=opts.LabelOpts(is_show=False),
        )
    )

    bar.overlap(line)
    return Grid().add(
        bar, opts.GridOpts(pos_left="5%", pos_right="20%"), is_control_axis_index=True
    )

In [13]:
# DraggablePageLayout 利用 Jquery 以及 Echarts 本身的 resize 功能，实现了可拖拽布局。使用步骤如下
# 1.指定 Page 布局
page = Page(layout=Page.DraggablePageLayout)

# 正常 render 图表
page.add(bar_datazoom_slider(), line_markpoint(), pie_rosetype(), grid_mutil_yaxis())
page.render()

'C:\\Users\\蓝尼玛\\render.html'

In [19]:
# 使用浏览器打开渲染后的 .html 文件，默认为 render.html。拖拉/调整图表位置和大小，当调整到一个适合
# 的布局时，点击左上方的 `Save Config` 按钮，下载 chart_config.json 配置文件，假设存放位置为
# ~/chart_config.json。再次渲染图表并指定其布局配置

# Warning: 请注释掉上面的的所有渲染代码，就是以下三行。因为 html 已经生成，并不需要再重新渲染一遍。
# page = Page(layout=Page.DraggablePageLayout)
# page.add(bar_datazoom_slider(), line_markpoint(), pie_rosetype(), grid_mutil_yaxis())
# page.render()

# render.html：第一步生成的原 html 文件
# chart_config.json：第二步下载的配置文件
# my_new_charts.html：新 html 文件路径
Page.save_resize_html("render.html", cfg_file="chart_config.json", dest="my_new_charts.html")

# 或者可以使用 json 数据
# cfg_dict 为 json 文件里面的内容
#Page.save_resize_html("render.html", cfg_dict="chart_config.json", dest="my_new_charts.html")

# Question：能否复用渲染模板？
# Answer: 可以的，渲染配置 json 数据中是以 chart_id 来作为一个图形的唯一标识符的，所以只需要在
# 第一次渲染的时候指定 chart_id 就可以啦。
# example:
# bar = bar_datazoom_slider()
# bar.chart_id = "chenjiandongx_is_an_awesome_boy"
# line = line_markpoint()
# line.chart_id = "chenjiandongx_is_an_amazing_boy"
# pie = pie_rosetype()
# pie.chart_id = "chenjiandongx_is_an_adorable_boy"
# 然后只要以后都按这个 chart_id 来渲染图表的时候，你的布局配置就可以复用啦。
# cat chart_config.json，会发现 chart_id 是固定的啦。
page.add(bar_datazoom_slider(), line_markpoint(), pie_rosetype())
page.render()

'C:\\Users\\蓝尼玛\\render.html'